In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
from keras.preprocessing.text import Tokenizer
from keras import models
from keras import layers
from keras import optimizers
import pickle

In [9]:
train = pickle.load(open("../pickle/train.pickle", "rb"))
val = pickle.load(open("../pickle/val.pickle", "rb"))
train

,tweet,target,lem
0,big night wcvb martha coakleycharlie baker de...,0,big night wcvb martha coakleycharlie baker deb...
1,not one bitch texting right now followers in...,0,not one bitch texting right now follower inch ...
2,going miss these bitches much far right she...,0,going miss these bitch much far right she rest...
3,hope shes larger than bird,0,hope shes larger than bird
4,all went through much shit together miss th...,0,all went through much shit together miss them ...
...,...,...,...
18582,the last atbat yankee stadium thanks for maki...,0,the last atbat yankee stadium thanks for makin...
18583,lmfaoooo yooo lost elevator pass now cant e...,0,lmfaoooo yooo lost elevator pas now cant even ...
18584,porn android iphone ipad sex xxx anal hardcore...,0,porn android iphone ipad sex xxx anal hardcore...
18585,just when thought justin bieber couldnt anym...,0,just when thought justin bieber couldnt anymor...


In [10]:
val

,tweet,target,lem
0,got missed call from bitch,0,got missed call from bitch
1,fucking with bad bitch you gone need some mon...,0,fucking with bad bitch you gone need some mone...
2,lol credit aint where near good but know th...,0,lol credit aint where near good but know the r...
3,wipe the cum out them faggot contact lens wi...,0,wipe the cum out them faggot contact lens wild...
4,niggas cheat they bitch and dont expect pay ...,0,nigga cheat they bitch and dont expect pay bac...
...,...,...,...
6191,lol now all need sum beacg bod abs pussy boy,0,lol now all need sum beacg bod ab pussy boy
6192,jus exsposin hoes whole different level,0,jus exsposin hoe whole different level
6193,might have play the tonight for black hoes y...,0,might have play the tonight for black hoe you ...
6194,dads side the family ghetto moms side th...,0,dad side the family ghetto mom side the family...


In [4]:
test = pd.concat([X_tt, y_tt], axis=1)
test = test.reset_index(drop=True)
test.head()

,lem,target
0,you cute you fancy bitch,0
1,dont buy pussy cuz it free bitch buy lac,0
2,such trash episode the walking dead tonight,0
3,big bitch sayin why eye all stomach bitch why ...,0
4,vice president biden his wife are florida toda...,0


In [5]:
train = train.drop(columns=['tweet'], axis=1)
train.head()

,target,lem
0,0,big night wcvb martha coakleycharlie baker deb...
1,0,not one bitch texting right now follower inch ...
2,0,going miss these bitch much far right she rest...
3,0,hope shes larger than bird
4,0,all went through much shit together miss them ...


In [7]:
from gensim.models import Word2Vec

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence
import multiprocessing
from sklearn import utils


In [8]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result

In [ ]:
all_x = pd.concat

In [ ]:
WWWW